# Create a Simple Reflex-Based Lunar Lander Agent

In this example, we will use Gymnasium, an environment to train agents via reinforcement learning (RL). We will not use RL here but just use the environment with a custom rule-based agent. 

## Install Gymnasium

The documentation for Gymnasium is available at https://gymnasium.farama.org/ 

Steps:
1. Create a new folder and open it with VS Code and install all needed Python Extensions in VS Code.
2. Create a new virtual environment (CTRL-Shift P Python Create Environment...)
3. I needed to install swig and the Python C++ headers on WSL2 via the terminal
    * `sudo apt install swig`
    * `sudo apt-get install python3-dev` 
4. Install gymnasium with the needed extras

In [59]:
%pip install gymnasium[box2d,classic_control]


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## The Lunar Lander Environment 

The documentation of the environment is available at: https://gymnasium.farama.org/environments/box2d/lunar_lander/

* Performance Measure: A reward of -100 or +100 points for crashing or landing safely respectively. We do not use 
  intermediate rewards here.

* Environment: This environment is a classic rocket trajectory optimization problem. A ship needs to land safely. The space is **continuous** with
  x and y coordinates in the range [-2.5, 2.5]. The landing pad is at coordinate (0,0).

* Actuators:  According to Pontryagin’s
  maximum principle, it is optimal to fire the engine at full throttle or turn it off. This is the reason why this environment has discrete actions: engine on or off. There are four discrete actions available:

    - 0: do nothing
    - 1: fire left orientation engine
    - 2: fire main engine
    - 3: fire right orientation engine

* Sensors: Each observation is an 8-dimensional vector: the coordinates of the lander in x & y, its linear velocities in x & y, its angle, its angular velocity, and two booleans that represent whether each leg is in contact with the ground or not.

Define an environment function based on the example in the documentation.

In [60]:
import gymnasium as gym

def run_episode(agent):
    # Initialise the environment
    env = gym.make("LunarLander-v3", render_mode="human")

    # Reset the environment to generate the first observation (use seed=42 in reset to get reproducible results)
    observation, info = env.reset()

    # run one episode (max. 1000 steps)
    for _ in range(1000):
        # call the agent function to select an action
        action = agent(observation)

        print (f"Obs: {observation} -> Action: {action}")

        # step: execute an action in the environment
        observation, reward, terminated, truncated, info = env.step(action)
    
        env.render()

        if terminated:
            print(f"Final Reward: {reward}")
            break
    
    env.close()
    return reward

## Example: A Random Agent

We ranomly return one of the actions. The environment accepts the integers 0-3.


In [61]:
import numpy as np


# define a random agent
def random_agent(observation): 
    return np.random.choice([0, 1, 2, 3], p=[0.25, 0.25, 0.25, 0.25])

run_episode(random_agent)

Obs: [-7.1907045e-05  1.4206916e+00 -7.3085218e-03  4.3428257e-01
  9.0213209e-05  1.6554941e-03  0.0000000e+00  0.0000000e+00] -> Action: 2
Obs: [-1.6899109e-04  1.4309193e+00 -9.6660415e-03  4.5456710e-01
  5.2757186e-05 -7.4863993e-04  0.0000000e+00  0.0000000e+00] -> Action: 1
Obs: [-3.5524368e-04  1.4405596e+00 -2.0853955e-02  4.2845747e-01
  2.2589364e-03  4.4127755e-02  0.0000000e+00  0.0000000e+00] -> Action: 0
Obs: [-5.4149627e-04  1.4496001e+00 -2.0860922e-02  4.0179011e-01
  4.4637173e-03  4.4100005e-02  0.0000000e+00  0.0000000e+00] -> Action: 0
Obs: [-7.2784425e-04  1.4580405e+00 -2.0867301e-02  3.7512165e-01
  6.6686426e-03  4.4102807e-02  0.0000000e+00  0.0000000e+00] -> Action: 2
Obs: [-1.1008263e-03  1.4667723e+00 -3.8654607e-02  3.8807386e-01
  7.9999510e-03  2.6628876e-02  0.0000000e+00  0.0000000e+00] -> Action: 0
Obs: [-1.4739990e-03  1.4749039e+00 -3.8658179e-02  3.6139825e-01
  9.3317963e-03  2.6639218e-02  0.0000000e+00  0.0000000e+00] -> Action: 0
Obs: [-0.0018

-100

## A Simple Reflex-Based Agent

To make the code easier to read, we use enumerations for actions (integers) and observations (index in the observation vector).

In [62]:
from enum import Enum

class Act(Enum):
    LEFT = 1
    RIGHT = 3
    MAIN = 2
    NO_OP = 0

class Obs(Enum):
    X = 0
    Y = 1
    VX = 2
    VY = 3
    ANGLE = 4
    ANGULAR_VELOCITY = 5
    LEFT_LEG_CONTACT = 6
    RIGHT_LEG_CONTACT = 7


In [63]:
def rocket_agent(observation):
    """A simple agent."""

    # run the main thruster, if the lander is falling too fast
    if observation[Obs.VY.value] < -.3:  
        return Act.MAIN.value

    return Act.NO_OP.value 

run_episode(rocket_agent)

Obs: [ 0.00169973  1.4114629   0.17214     0.02412427 -0.00196268 -0.03899228
  0.          0.        ] -> Action: 0
Obs: [ 0.00339928  1.4114282   0.17190754 -0.00155153 -0.00388947 -0.03853852
  0.          0.        ] -> Action: 0
Obs: [ 0.00509882  1.4107935   0.17191367 -0.02820851 -0.00581511 -0.03851638
  0.          0.        ] -> Action: 0
Obs: [ 0.00679855  1.4095589   0.17191938 -0.05488018 -0.00774048 -0.03851127
  0.          0.        ] -> Action: 0
Obs: [ 0.00849829  1.4077243   0.17192508 -0.08154889 -0.00966558 -0.03850527
  0.          0.        ] -> Action: 0
Obs: [ 0.01019812  1.4052896   0.17193078 -0.10821744 -0.01159035 -0.0384992
  0.          0.        ] -> Action: 0
Obs: [ 0.01189795  1.402255    0.17193644 -0.13488597 -0.01351482 -0.03849318
  0.          0.        ] -> Action: 0
Obs: [ 0.01359787  1.3986205   0.17194211 -0.16155456 -0.015439   -0.03848723
  0.          0.        ] -> Action: 0
Obs: [ 0.01529789  1.394386    0.17194776 -0.18822321 -0.01736287

100

## Evaluating the Agent

Run the agent on 100 problems and report the average reward.

In [64]:
import numpy as np

def run_episode_test(agent):
    # Initialise the environment
    env = gym.make("LunarLander-v3", render_mode=None)

    # Reset the environment to generate the first observation
    observation, info = env.reset()

    # run one episode (max. 1000 steps)
    for _ in range(1000):
        # call the agent to select an action
        action = agent(observation)

        # step (transition) through the environment with the action
        observation, reward, terminated, truncated, info = env.step(action)
      
        if terminated:
            break
    
    env.close()
    return reward

def run_episodes(agent, n=1000):
    return [run_episode_test(agent) for _ in range(n)]


rewards = run_episodes(rocket_agent)
print(rewards)

print(f"Average reward: {np.average(rewards)}")
print(f"Success rate: {np.sum(np.array(rewards) == 100)}/{len(rewards)}")

[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -10

This is not great!


## Implement A Better Reflex-Based Agent

Build a better that uses its right and left thrusters to land the craft (more) safely. Test your agent function using 100 problems.

In [65]:
# Code goes here